In [133]:
from random import choices
from datetime import datetime
from datetime import date
from tinydb import TinyDB, Query
import sys
import json
import logging

logging.basicConfig()
logger = logging.getLogger()
logger.setLevel(logging.INFO)

date_format = "%d.%m.%Y"
dbname = "colleagues.json"
logger.info("Using file %s", dbname)
db = TinyDB(dbname)


INFO:root:Using file colleagues.json


In [134]:
class InputHelper:    
    def getNextCoffeeDate():
        coffee_date = None
        while not coffee_date:
            coffee_date_text = input("Termin setzen am dd.mm.YYYY!")
            if(len(coffee_date_text) > 0):
                try:
                    temp_coffee_date = datetime.strptime(coffee_date_text, date_format)

                    if((temp_coffee_date - datetime.today()).days >= 0):
                        coffee_date = temp_coffee_date
                    else:
                        logger.info("%s is in the past", coffee_date_text)

                except ValueError:
                    logger.info("Value %s is not a valid date", coffee_date_text)
            else:
                break

        return coffee_date

In [135]:
class Employee:
    def __init__(self, name, company, willing, last_coffee):
        self.name = name
        self.company = company
        self.willing = willing
        self.last_coffee = last_coffee

    def weight(self):
        temp_weight = (datetime.today() - datetime.strptime(self.last_coffee, date_format)).days
        
        if(self.company != "Verimi"):
            temp_weight = temp_weight * 0.7    
        if(self.willing):
            temp_weight = temp_weight * 1.5

        return temp_weight

    def days_since_last_coffee(self):
        return (datetime.today() - datetime.strptime(self.last_coffee, date_format)).days


In [136]:
# db.truncate()

# db.insert(Employee("Patrick", "Verimi", True, "28.10.2020").__dict__)
# db.insert(Employee("Liang", "Verimi", False, "29.10.2020").__dict__)
# db.insert(Employee("Sukhmeet", "Verimi", True, "29.10.2020").__dict__)

# db.insert(Employee("Jan", "Verimi", True, "02.11.2020").__dict__)
# db.insert(Employee("Hadi", "Verimi", True, "30.10.2020").__dict__)
# db.insert(Employee("Manju", "Verimi", True, "28.10.2020").__dict__)
# db.insert(Employee("Oleh", "Verimi", True, "23.10.2020").__dict__)
# db.insert(Employee("Svitlana", "Verimi", True, "29.10.2020").__dict__)

# db.insert(Employee("Gurpreet", "Verimi", True, "23.10.2020").__dict__)
# db.insert(Employee("Chidubem", "Verimi", True, "28.10.2020").__dict__)
# db.insert(Employee("Bharat", "Verimi", True, "23.10.2020").__dict__)
# db.insert(Employee("Tapan", "Verimi", True, "30.10.2020").__dict__)
# db.insert(Employee("Tarun", "Verimi", False, "30.10.2020").__dict__)

# db.insert(Employee("Sandor", "EPAM", True, "29.10.2020").__dict__)
# db.insert(Employee("Andras B.", "EPAM", True, "23.09.2020").__dict__)
# db.insert(Employee("Andrea", "EPAM", False, "26.10.2020").__dict__)
# db.insert(Employee("Szilvia", "EPAM", False, "30.10.2020").__dict__)

# db.insert(Employee("Andras S.", "EPAM", True, "02.11.2020").__dict__)
# db.insert(Employee("Peter", "EPAM", True, "30.10.2020").__dict__)
# db.insert(Employee("Mirjam", "EPAM", True, "26.10.2020").__dict__)
# db.insert(Employee("Oleks", "EPAM", True, "02.11.2020").__dict__)

# db.insert(Employee("Lajos", "EPAM", True, "26.10.2020").__dict__)
# db.insert(Employee("Miklos", "EPAM", True, "23.09.2020").__dict__)
# db.insert(Employee("Bernat", "EPAM", True, "26.10.2020").__dict__)
# db.insert(Employee("Biju", "Verimi", True, "30.10.2020").__dict__)
# db.insert(Employee("Mohamed", "Verimi", True, "02.11.2020").__dict__)
# db.insert(Employee("Marius", "Verimi", True, "23.10.2020").__dict__)
# db.insert(Employee("Thierry", "Verimi", True, "30.10.2020").__dict__)


In [137]:
names = []
weights = []
winners = []

for entry in db.all():
    e = Employee(**entry)
    if not (int(e.days_since_last_coffee()) <= 0): # invited already
        names.append(e.name)
        weights.append(e.weight())
        logger.info('%s: %s', e.name, e.weight())

if(len(names) > 4):
    while(len(winners) < 4):
        next = choices(names, weights = weights, k = 1)
        if(next[0] not in winners):
            winners.append(next[0])
else:
    logger.warn("Not enough coffee candidates")
    sys.exit()

print("")
print("########################################################################")
print("Have coffee with")
print(winners)
print("########################################################################")
print("")

coffee_date = InputHelper.getNextCoffeeDate()
if coffee_date:
    for e_name in winners:
        date_for_db = coffee_date.strftime(date_format)
        db.update({'last_coffee': date_for_db}, Query().name == e_name)
        logger.info("Coffee Date %s stored to database for %s", date_for_db, e_name)
else:
    logger.warn("Coffee date was not updated")

INFO:root:Oleh: 4.5
INFO:root:Gurpreet: 4.5
INFO:root:Bharat: 4.5
INFO:root:Andras B.: 34.65
INFO:root:Miklos: 34.65
INFO:root:Marius: 4.5

########################################################################
Have coffee with
['Marius', 'Andras B.', 'Miklos', 'Bharat']
########################################################################

